In [1]:
import numpy as np
import math
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from scipy.stats import ttest_ind, ttest_rel

In [48]:
x, y = load_iris(return_X_y=True)

In [49]:
y = y.reshape(y.shape[0], 1)

In [50]:
y.shape

(150, 1)

In [51]:
x = np.concatenate((np.ones((x.shape[0],1), dtype=np.float64), x), axis=1)

In [52]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7)

In [2]:
def softmax(theta):
    """
    Returns a vector, the softmax of the vector eta.
    """
    exps = np.exp(theta - np.max(theta))
    return exps / np.sum(exps, axis=0)

In [122]:
theta = np.random.randn(3, x_train.shape[1])  # k x (n+1)

In [123]:
theta.shape

(3, 5)

In [3]:
def grad_nll(x, y, p, theta, lambd):
    """
    Returns the gradient of the negative log-likelihood function w.r.t theta_p.
    
    Args:
        x: (m, n + 1) matrix
        y: (m, 1) vector
        theta: (k, n+1) matrix
    """
    m = x.shape[0]
    n = x.shape[1]
    
    cur_sum = np.zeros(n, dtype=np.float64)
    for i in range(m):
        cur_sum += x[i] * ((y[i] == p) - softmax(np.dot(x[i], theta))[p])
    
    return -cur_sum / m #+ lambd * theta.T[p]

In [4]:
def gradient_descent(theta, x, y, iterations=1000, it_print=100, alpha=0.01, lambd=1):
    """
    Performs gradient descent.
    
    Args:
        theta: (n + 1, k) vector
        x: (m, n+1) vector
        y: (m, 1) vector
        iterations: number of iterations to perform
        it_print: number of iterations to print loss after
        alpha: learning rate
        lambda: regularization parameter
        
    """
    m = x.shape[0]
    n = x.shape[1]
    k = theta.shape[1]
    
    x_norm = np.sum(x, axis=0)
    x = x / x_norm
    
    for i in range(iterations):
        grad = np.array([])
        for p in range(k):
            grad = np.append(grad, grad_nll(x, y, p, theta, lambd))
        grad = grad.reshape(theta.shape)
        theta -= alpha * grad
        
    return theta

In [126]:
tf = gradient_descent(theta.T, x_train, y_train, iterations=300, alpha=0.1)

In [127]:
probabilities = softmax(np.dot(x_test, tf))

/home/rahul/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


In [128]:
predictions = np.argmax(probabilities, axis=1)

In [129]:
predictions = predictions.reshape((predictions.shape[0], 1))

In [130]:
y_test.shape == predictions.shape

True

In [131]:
right_predictions = np.sum(y_test == predictions)

In [132]:
print('Accuracy = {0}'.format(right_predictions / len(predictions)))

Accuracy = 0.4888888888888889


In [61]:
def run_experiment(x, y, alpha=0.1, n_iter=100):
    np.random.shuffle(x)
    
    #x_norm = np.max(x, axis=0)
    #x /= x_norm
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7)
    m = y.shape[0]
    
    k = len(np.unique(y))
    L = (k - 1) / (k * m) * np.linalg.norm(x_train)
    print('Learning rate =', 1/L)
    
    k = len(np.unique(y))
    theta = np.random.randn(x_train.shape[1], k)
    
    tf = gradient_descent(theta, x_train, y_train, iterations=n_iter, alpha=0.1)
    probabilities = softmax(np.dot(x_test, tf))
    predictions = np.argmax(probabilities, axis=1)
    predictions = predictions.reshape((predictions.shape[0], 1))
    right_predictions = np.sum(y_test == predictions)
    print('Normal Accuracy = {0}'.format(right_predictions / len(predictions)))
    acc1 = right_predictions / len(predictions)
    
    tf = gradient_descent(theta, x_train, y_train, iterations=n_iter, alpha=1/L)
    probabilities = softmax(np.dot(x_test, tf))
    predictions = np.argmax(probabilities, axis=1)
    predictions = predictions.reshape((predictions.shape[0], 1))
    right_predictions = np.sum(y_test == predictions)
    print('New Accuracy = {0}'.format(right_predictions / len(predictions)))
    acc2 = right_predictions / len(predictions)
    
    return acc1, acc2

In [ ]:
acc1 = []
acc2 = []

for i in range(30):
    a1, a2 = run_experiment(x, y, n_iter=80)
    acc1.append(a1)
    acc2.append(a2)

print('Normal accuracy:', np.mean(acc1))
print('New accuracy:', np.mean(acc2))

In [74]:
ttest_rel(acc1, acc2)

Ttest_relResult(statistic=0.6796393425278456, pvalue=0.5007471819619884)

In [38]:
run_experiment(x, y, n_iter=100)

Learning rate = 95.673302709263
Normal Accuracy = 0.4444444444444444
New Accuracy = 0.4888888888888889


## digits data

In [6]:
from sklearn.datasets import load_digits
import pandas as pd

In [55]:
x, y = load_digits(return_X_y=True)

In [56]:
y = y.reshape(y.shape[0], 1)

In [57]:
x = np.concatenate((np.ones((x.shape[0],1), dtype=np.float64), x), axis=1)

In [62]:
run_experiment(x, y)

/home/rahul/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/rahul/.local/lib/python3.5/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


Learning rate = 0.9075378817851076
Normal Accuracy = 0.1111111111111111


KeyboardInterrupt: 

In [40]:
from sklearn.linear_model import LogisticRegression

In [58]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')

In [59]:
#x_norm = np.max(x, axis=0)
#x /= x_norm

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7)

/home/rahul/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [60]:
lr.fit(x_train, y_train)
print(lr.score(x_test, y_test))

/home/rahul/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9629629629629629


/home/rahul/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
